In [ ]:
import sys
print(sys.executable)

In [ ]:

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

In [ ]:

fname = "../data/catalog_data_4FGL_DR4.h5"

with h5py.File(fname, "r") as f:
    # read attributes
    E0 = f.attrs["E0"]
    
    # read assoc data (fuentes conocidas - datos de entrenamiento)
    alpha_as = f["assoc"]["alpha"][:]
    beta_as = f["assoc"]["beta"][:]
    flux_as = f["assoc"]["flux"][:]
    
    # read unassoc data (candidatos a materia oscura)
    alpha_unas = f["unas"]["alpha"][:]
    beta_unas = f["unas"]["beta"][:]
    flux_unas = f["unas"]["flux"][:]

In [ ]:
print('E0 (energy): attribute value', E0)
print('UNAS - alpha, beta & flux shape: ', alpha_unas.shape, beta_unas.shape, flux_unas.shape)
print('AS - alpha, beta & flux shape: ', alpha_as.shape, beta_as.shape, flux_as.shape)

In [ ]:

# Convert to pandas DataFrame
df_as = pd.DataFrame({
    'alpha': alpha_as,
    'beta': beta_as,
    'flux': flux_as
})

df_unas = pd.DataFrame({
    'alpha': alpha_unas,
    'beta': beta_unas,
    'flux': flux_unas
})

print(f"Datos cargados exitosamente:")
print(f"- Fuentes asociadas (entrenamiento): {len(df_as)}")
print(f"- Fuentes no asociadas (predicción): {len(df_unas)}")
print(f"- Energía de referencia E0: {E0}")

In [ ]:
# Double check the data conversion
df_as.shape[0] == len(alpha_as)
df_unas.shape[0] == len(alpha_unas)

In [ ]:
print(df_as.shape)
df_as.head()

In [ ]:
print(df_unas.shape)
df_unas.head()

In [ ]:
print('Associated sources: ')
df_as.describe()

In [ ]:
print('\nUnassociated sources:')
df_unas.describe()

In [ ]:
print("\nEstadísticas descriptivas - Fuentes Asociadas:")
print(df_as.describe())

print("\nEstadísticas descriptivas - Fuentes No Asociadas:")
print(df_unas.describe())

# Verificar calidad de datos
print(f"\nCalidad de datos:")
print(f"- Valores nulos en asociadas: {df_as.isnull().sum().sum()}")
print(f"- Valores nulos en no asociadas: {df_unas.isnull().sum().sum()}")
print(f"- Valores infinitos en asociadas: {np.isinf(df_as.values).sum()}")
print(f"- Valores infinitos en no asociadas: {np.isinf(df_unas.values).sum()}")

In [ ]:
# Plot the data the same way to check the distributions
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

nbins = 15
unas_color = "#3e4550"
as_color = "#c2d9c7"

fig.suptitle(r"4FGL DR4 data at E$_0$ = {:.0f} MeV".format(E0), fontsize=18)

# Alpha
axs[0].hist(df_as['alpha'], bins=nbins, range=[-2, 4], color=as_color, edgecolor="white", lw=2, density=True, label="Assoc")
axs[0].hist(df_unas['alpha'], bins=nbins, range=[-2, 4], histtype='step', edgecolor=unas_color, lw=2, density=True, label="Unas")
axs[0].set_xlabel(r"$\alpha$")
axs[0].set_ylabel("Density")
axs[0].legend()
axs[0].grid(0)
axs[0].set_ylim(0, 1.0)

# Beta
axs[1].hist(df_as['beta'], bins=nbins, range=[-0.25, 1.0], color=as_color, edgecolor="white", lw=2, density=True, label="Assoc")
axs[1].hist(df_unas['beta'], bins=nbins, range=[-0.25, 1.0], histtype='step', edgecolor=unas_color, lw=2, density=True, label="Unas")
axs[1].set_xlabel(r"$\beta$")
axs[1].set_ylabel("Density")
axs[1].legend()
axs[1].grid(0)

# Flux
axs[2].hist(df_as['flux'], bins=nbins, range=[-12, -6], color=as_color, edgecolor="white", lw=2, density=True, label="Assoc")
axs[2].hist(df_unas['flux'], bins=nbins, range=[-12, -6], histtype='step', edgecolor=unas_color, lw=2, density=True, label="Unas")
axs[2].set_xlabel(r"$\log_{10} \phi$")
axs[2].set_ylabel("Density")
axs[2].legend()
axs[2].grid(0)

plt.tight_layout()
plt.show()

In [ ]:
# Visualización de distribuciones
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

features = ['alpha', 'beta', 'flux']
for i, feature in enumerate(features):
    # Histogramas comparativos
    axes[0, i].hist(df_as[feature], bins=50, alpha=0.7, label='Asociadas', density=True, color='blue')
    axes[0, i].hist(df_unas[feature], bins=50, alpha=0.7, label='No Asociadas', density=True, color='red')
    axes[0, i].set_xlabel(feature)
    axes[0, i].set_ylabel('Densidad')
    axes[0, i].legend()
    axes[0, i].set_title(f'Distribución de {feature}')
    axes[0, i].grid(True, alpha=0.3)
    
    
    # Box plots comparativos
    data_to_plot = [df_as[feature], df_unas[feature]]
    box_plot = axes[1, i].boxplot(data_to_plot, labels=['Asociadas', 'No Asociadas'], patch_artist=True)
    box_plot['boxes'][0].set_facecolor('lightblue')
    box_plot['boxes'][1].set_facecolor('lightcoral')
    axes[1, i].set_ylabel(feature)
    axes[1, i].set_title(f'Box Plot de {feature}')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## PREPROCESAMIENTO DE DATOS

In [ ]:

# Seleccionar features
features = ['alpha', 'beta', 'flux']
X = df_as[features].values

In [ ]:
# Limpiar datos: remover infinitos y NaN
print(f"Datos originales: {X.shape[0]} muestras")
mask = np.isfinite(X).all(axis=1)
X_clean = X[mask]
print(f"Datos después de limpiar infinitos/NaN: {X_clean.shape[0]} muestras")
print(f"Muestras quitadas: {X.shape[0] - X_clean.shape[0]}")

In [ ]:

# División de datos
X_train, X_temp = train_test_split(X_clean, test_size=0.4, random_state=42)
X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=42)


print(f"\nDivisión de datos:")
print(f"- Entrenamiento: {X_train.shape[0]} muestras ({X_train.shape[0]/X_clean.shape[0]*100:.1f}%)")
print(f"- Validación: {X_val.shape[0]} muestras ({X_val.shape[0]/X_clean.shape[0]*100:.1f}%)")
print(f"- Test: {X_test.shape[0]} muestras ({X_test.shape[0]/X_clean.shape[0]*100:.1f}%)")

In [ ]:

# Escalado de datos - usando RobustScaler para mayor robustez ante outliers
scaler = RobustScaler()  # Menos sensible a outliers que StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"\nEscalado completado usando RobustScaler")
print(f"Estadísticas de datos escalados (train):")
print(f"- Media: {np.mean(X_train_scaled, axis=0)}")
print(f"- Std: {np.std(X_train_scaled, axis=0)}")

# Train the model - 3F OneClassSVM

In [ ]:

# Grid de hiperparámetros ampliado
# nu_values = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1]
# gamma_values = ['scale', 'auto'] + list(np.logspace(-4, 1, 8))

nu_values = [0.002]
gamma_values = [0.02]

print(f"Probando {len(nu_values)} valores de nu y {len(gamma_values)} valores de gamma")
print(f"Total de combinaciones: {len(nu_values) * len(gamma_values)}")

best_model = None
best_params = {}
best_score = np.inf
results = []

total_combinations = len(nu_values) * len(gamma_values)
current_combination = 0

for nu in nu_values:
    for gamma in gamma_values:
        current_combination += 1
        
        # Mostrar progreso cada 10 iteraciones
        if current_combination % 10 == 0 or current_combination == total_combinations:
            print(f"Progreso: {current_combination}/{total_combinations} ({current_combination/total_combinations*100:.1f}%)")
        
        model = OneClassSVM(kernel='rbf', nu=nu, gamma=gamma)
        model.fit(X_train_scaled)
        
        # Predicciones en validación
        preds_val = model.predict(X_val_scaled)
        decision_scores = model.decision_function(X_val_scaled)
        
        # Métricas detalladas
        n_outliers = np.sum(preds_val == -1)
        outlier_ratio = n_outliers / len(preds_val)
        mean_score = np.mean(decision_scores)
        std_score = np.std(decision_scores)
        
        results.append({
            'nu': nu, 
            'gamma': gamma, 
            'outliers': n_outliers,
            'outlier_ratio': outlier_ratio,
            'mean_score': mean_score,
            'std_score': std_score
        })
        
        # Criterio de selección mejorado
        target_ratio = nu  # nu aproxima el ratio esperado de outliers
        ratio_penalty = abs(outlier_ratio - target_ratio)
        score_quality = abs(mean_score)  # Preferimos scores más extremos
        combined_score = ratio_penalty + (1 - score_quality) / 10
        
        if combined_score < best_score:
            best_score = combined_score
            best_model = model
            best_params = {'nu': nu, 'gamma': gamma}

# Convertir resultados a DataFrame para análisis
results_df = pd.DataFrame(results)

In [ ]:
print(f"\nMEJORES PARÁMETROS ENCONTRADOS")
print(f"Nu: {best_params['nu']}")
print(f"Gamma: {best_params['gamma']}")

# Mostrar estadísticas del mejor modelo
best_result = results_df[(results_df['nu'] == best_params['nu']) & 
                        (results_df['gamma'] == best_params['gamma'])].iloc[0]

print(f"\nEstadísticas del mejor modelo en validación:")
print(f"- Outliers detectados: {best_result['outliers']} ({best_result['outlier_ratio']*100:.2f}%)")
print(f"- Score promedio: {best_result['mean_score']:.4f} ± {best_result['std_score']:.4f}")


In [ ]:

# Entrenamiento final del modelo con los mejores hiperparámetros
final_model = OneClassSVM(kernel='rbf', nu=best_params['nu'], gamma=best_params['gamma'])
final_model.fit(X_train_scaled)

In [ ]:

# Predicciones en test
test_preds = final_model.predict(X_test_scaled)
test_scores = final_model.decision_function(X_test_scaled)

# Estadísticas básicas
n_inliers_test = np.sum(test_preds == 1)
n_outliers_test = np.sum(test_preds == -1)

print(f"Resultados en datos de test:")
print(f"- Inliers: {n_inliers_test} ({n_inliers_test/len(test_preds)*100:.2f}%)")
print(f"- Outliers: {n_outliers_test} ({n_outliers_test/len(test_preds)*100:.2f}%)")
print(f"- Score promedio: {np.mean(test_scores):.4f} ± {np.std(test_scores):.4f}")
print(f"- Score mínimo: {np.min(test_scores):.4f}")
print(f"- Score máximo: {np.max(test_scores):.4f}")

In [ ]:
from itertools import combinations

# Nombres cortos para cada dimensión en escala
feature_names = ['Alpha', 'Beta', 'Flux']

# Todas las combinaciones de pares (índices) de las 3 características
feature_pairs = list(combinations(range(len(feature_names)), 2))
n_pairs = len(feature_pairs)

# Definimos el layout del grid
cols = 3
rows = int(np.ceil(n_pairs / cols))
fig, axes = plt.subplots(rows, cols, figsize=(18, 6*rows))

# Asegurarnos de que axes siempre sea 2D
if rows == 1:
    axes = axes.reshape(1, -1)
if cols == 1:
    axes = axes.reshape(-1, 1)

# Máscaras de inliers/outliers sobre los datos ASOC de test
inliers_mask_asoc = (test_preds == 1)
outliers_mask_asoc = (test_preds == -1)

# Recorremos cada par de características para dibujar subplots
for i, (idx1, idx2) in enumerate(feature_pairs):
    row = i // cols
    col = i % cols
    ax = axes[row, col]
    
    # La dimensión que no se muestra en este par
    other_dim = [j for j in range(3) if j not in (idx1, idx2)][0]
    # Fijamos esa dimensión en su mediana (sobre X_test_scaled)
    med_val = np.median(X_test_scaled[:, other_dim])
    
    # Rango de valores en idx1 e idx2 para la malla 2D
    x_min = X_test_scaled[:, idx1].min() - 0.5
    x_max = X_test_scaled[:, idx1].max() + 0.5
    y_min = X_test_scaled[:, idx2].min() - 0.5
    y_max = X_test_scaled[:, idx2].max() + 0.5
    
    # Creamos la malla 2D
    res = 150
    xx = np.linspace(x_min, x_max, res)
    yy = np.linspace(y_min, y_max, res)
    XX, YY = np.meshgrid(xx, yy)
    
    # Preparamos el grid completo (res^2 × 3)
    grid = np.zeros((res*res, 3))
    grid[:, idx1] = XX.ravel()
    grid[:, idx2] = YY.ravel()
    grid[:, other_dim] = med_val
    
    # Computar decision_function en cada punto de la malla
    Z = final_model.decision_function(grid).reshape((res, res))
    
    # Ajustar niveles para contourf
    z_min = Z.min()
    z_max = 0  # la frontera OCSVM está en decision_function=0
    if z_min >= z_max:
        z_max = z_min + 1e-3
    
    levels = np.linspace(z_min, z_max, 8)
    
    # Dibujar contorno relleno y frontera
    cf = ax.contourf(XX, YY, Z, levels=levels, cmap='RdYlBu_r', alpha=0.6)
    ax.contour(XX, YY, Z, levels=[0], colors='firebrick', linewidths=2, linestyles='--')
    
    # Scatter de los inliers de ASOC en test
    ax.scatter(
        X_test_scaled[inliers_mask_asoc, idx1], 
        X_test_scaled[inliers_mask_asoc, idx2],
        c='navy', s=20, alpha=0.7,
        label=f'Inliers (ASOC): {np.sum(inliers_mask_asoc)}'
    )
    # Scatter de los outliers de ASOC en test
    if np.any(outliers_mask_asoc):
        ax.scatter(
            X_test_scaled[outliers_mask_asoc, idx1],
            X_test_scaled[outliers_mask_asoc, idx2],
            c='red', marker='D', s=50, alpha=1.0,
            label=f'Outliers (ASOC): {np.sum(outliers_mask_asoc)}'
        )
    
    # Etiquetas y título
    ax.set_xlabel(feature_names[idx1])
    ax.set_ylabel(feature_names[idx2])
    gamma_val = final_model.gamma if hasattr(final_model, 'gamma') else final_model._gamma
    nu_val = final_model.nu
    ax.set_title(
        f"{feature_names[idx1]} vs {feature_names[idx2]}\n"
        f"ν={nu_val}, γ={gamma_val:.2e} | "
        f"Outliers: {np.sum(outliers_mask_asoc)}/{len(X_test_scaled)}"
    )
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=8)

# Ocultar subplots vacíos si los hay
total_subplots = rows * cols
for j in range(n_pairs, total_subplots):
    r = j // cols
    c = j % cols
    axes[r, c].set_visible(False)

plt.tight_layout()
plt.show()

# Información adicional del modelo
print(f"\nOne-Class SVM (ASOC test):")
print(f"  • Kernel : {final_model.kernel}")
print(f"  • Gamma  : {gamma_val:.2e}")
print(f"  • Nu     : {nu_val}")
print(f"  • Soportes: {final_model.n_support_[0]}")
print(f"  • Outliers detectados en ASOC test: {np.sum(outliers_mask_asoc)}/"
      f"{len(X_test_scaled)}  ({100*np.sum(outliers_mask_asoc)/len(X_test_scaled):.2f}%)")


In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # necesario para la proyección 3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Inliers de ASOC test (gris tenue)
inliers_mask_asoc = (test_preds == 1)
ax.scatter(
    X_test_scaled[inliers_mask_asoc, 0],
    X_test_scaled[inliers_mask_asoc, 1],
    X_test_scaled[inliers_mask_asoc, 2],
    c='purple', s=12, alpha=0.5, label=f'ASOC Inliers: {np.sum(inliers_mask_asoc)}'
)

# Outliers de ASOC test (rojo)
outliers_mask_asoc = (test_preds == -1)
if np.any(outliers_mask_asoc):
    ax.scatter(
        X_test_scaled[outliers_mask_asoc, 0],
        X_test_scaled[outliers_mask_asoc, 1],
        X_test_scaled[outliers_mask_asoc, 2],
        c='red', marker='x', s=60, alpha=0.8,
        label=f'ASOC Outliers: {np.sum(outliers_mask_asoc)}'
    )

# Etiquetas y título
ax.set_xlabel('Alpha (escalado)')
ax.set_ylabel('Beta (escalado)')
ax.set_zlabel('Flux (escalado)')
gamma_val = final_model.gamma if hasattr(final_model, 'gamma') else final_model._gamma
nu_val = final_model.nu
ax.set_title(f"3D ASOC Test\nν={nu_val}, γ={gamma_val:.2e}")
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


# Predecir sobre unassociated data (unas)

In [ ]:

# Preparar datos de fuentes no asociadas
features = ['alpha', 'beta', 'flux']
X_unas = df_unas[features].values

print(f"Fuentes no asociadas originales: {len(X_unas)}")


In [ ]:

# Limpiar datos no asociados
mask_unas = np.isfinite(X_unas).all(axis=1)
X_unas_clean = X_unas[mask_unas]
clean_indices = np.where(mask_unas)[0]

print(f"Fuentes no asociadas válidas: {len(X_unas_clean)}")
print(f"Fuentes quitadas por datos inválidos: {len(X_unas) - len(X_unas_clean)}")

In [ ]:

# Escalar datos usando el mismo scaler del entrenamiento
X_unas_scaled = scaler.transform(X_unas_clean)

# Realizar predicciones
unas_preds = final_model.predict(X_unas_scaled)
unas_scores = final_model.decision_function(X_unas_scaled)

In [ ]:

# get predictions + decision scores
preds = best_model.predict(X_val_scaled)
decision_scores = best_model.decision_function(X_val_scaled)

# index maxs
inlier_idx = preds == 1
outlier_idx = preds == -1

In [ ]:

# Análisis de resultados
n_normal = np.sum(unas_preds == 1)
n_anomalous = np.sum(unas_preds == -1)

print(f"\nResultados en fuentes no asociadas:")
print(f"- Fuentes normales: {n_normal} ({n_normal/len(unas_preds)*100:.2f}%)")
print(f"- Candidatos anómalos: {n_anomalous} ({n_anomalous/len(unas_preds)*100:.2f}%)")

# Estadísticas de scores
print(f"\nEstadísticas de scores de anomalía:")
print(f"- Score promedio: {np.mean(unas_scores):.4f} ± {np.std(unas_scores):.4f}")
print(f"- Score más anómalo: {np.min(unas_scores):.4f}")
print(f"- Score menos anómalo: {np.max(unas_scores):.4f}")


## Análisis de candidatos

In [ ]:

# Crear DataFrame con resultados completos
results_unas = df_unas.iloc[clean_indices].copy()
results_unas['prediction'] = unas_preds
results_unas['anomaly_score'] = unas_scores
results_unas['is_candidate'] = unas_preds == -1

# Filtrar candidatos anómalos
candidates = results_unas[results_unas['is_candidate']].copy()
candidates_sorted = candidates.sort_values('anomaly_score').reset_index(drop=True)

print(f"ANÁLISIS DE CANDIDATOS A MATERIA OSCURA")
print(f"Total de candidatos encontrados: {len(candidates)}")

if len(candidates) > 0:
    print(f"\nCaracterísticas de los candidatos:")
    print(f"- Alpha promedio: {candidates['alpha'].mean():.4f} ± {candidates['alpha'].std():.4f}")
    print(f"- Beta promedio: {candidates['beta'].mean():.4f} ± {candidates['beta'].std():.4f}")
    print(f"- Flux promedio: {candidates['flux'].mean():.2e} ± {candidates['flux'].std():.2e}")
    print(f"- Score de anomalía promedio: {candidates['anomaly_score'].mean():.4f}")
    
    print(f"\nTop 10 candidatos más anómalos:")
    top_candidates = candidates_sorted[['alpha', 'beta', 'flux', 'anomaly_score']].head(10)
    print(top_candidates.to_string(index=True, float_format='%.4f'))
    
    print(f"\nComparación con fuentes asociadas:")
    print(f"- Alpha: Candidatos={candidates['alpha'].mean():.4f}, Asociadas={df_as['alpha'].mean():.4f}")
    print(f"- Beta: Candidatos={candidates['beta'].mean():.4f}, Asociadas={df_as['beta'].mean():.4f}")
    print(f"- Flux: Candidatos={candidates['flux'].mean():.2e}, Asociadas={df_as['flux'].mean():.2e}")


In [ ]:
feature_names = ['Alpha', 'Beta', 'Flux']
feature_pairs = list(combinations(range(len(feature_names)), 2))
n_pairs = len(feature_pairs)

cols = 3
rows = int(np.ceil(n_pairs / cols))
fig, axes = plt.subplots(rows, cols, figsize=(18, 6*rows))

if rows == 1:
    axes = axes.reshape(1, -1)
if cols == 1:
    axes = axes.reshape(-1, 1)

# Máscaras de inliers/outliers en UNAS
inliers_mask_unas = (unas_preds == 1)
outliers_mask_unas = (unas_preds == -1)

for i, (idx1, idx2) in enumerate(feature_pairs):
    row = i // cols
    col = i % cols
    ax = axes[row, col]
    
    other_dim = [j for j in range(3) if j not in (idx1, idx2)][0]
    med_val = np.median(X_unas_scaled[:, other_dim])
    
    x_min = X_unas_scaled[:, idx1].min() - 0.5
    x_max = X_unas_scaled[:, idx1].max() + 0.5
    y_min = X_unas_scaled[:, idx2].min() - 0.5
    y_max = X_unas_scaled[:, idx2].max() + 0.5
    
    res = 150
    xx = np.linspace(x_min, x_max, res)
    yy = np.linspace(y_min, y_max, res)
    XX, YY = np.meshgrid(xx, yy)
    
    grid = np.zeros((res*res, 3))
    grid[:, idx1] = XX.ravel()
    grid[:, idx2] = YY.ravel()
    grid[:, other_dim] = med_val
    
    Z = final_model.decision_function(grid).reshape((res, res))
    z_min = Z.min()
    z_max = 0
    if z_min >= z_max:
        z_max = z_min + 1e-3
    levels = np.linspace(z_min, z_max, 8)
    
    cf = ax.contourf(XX, YY, Z, levels=levels, cmap='RdYlBu_r', alpha=0.6)
    ax.contour(XX, YY, Z, levels=[0], colors='firebrick', linewidths=2, linestyles='--')
    
    # Puntos UNAS inliers
    ax.scatter(
        X_unas_scaled[inliers_mask_unas, idx1],
        X_unas_scaled[inliers_mask_unas, idx2],
        c='royalblue', s=25, alpha=0.5,
        label=f'UNAS Inliers: {np.sum(inliers_mask_unas)}'
    )
    # Puntos UNAS outliers (candidatos)
    if np.any(outliers_mask_unas):
        ax.scatter(
            X_unas_scaled[outliers_mask_unas, idx1],
            X_unas_scaled[outliers_mask_unas, idx2],
            c='darkred', marker='^', s=80, alpha=0.9,
            label=f'UNAS Outliers: {np.sum(outliers_mask_unas)}'
        )
    
    ax.set_xlabel(feature_names[idx1])
    ax.set_ylabel(feature_names[idx2])
    gamma_val = final_model.gamma if hasattr(final_model, 'gamma') else final_model._gamma
    nu_val = final_model.nu
    ax.set_title(
        f"{feature_names[idx1]} vs {feature_names[idx2]}\n"
        f"ν={nu_val}, γ={gamma_val:.2e} | "
        f"Outliers UNAS: {np.sum(outliers_mask_unas)}/{len(X_unas_scaled)}"
    )
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=8)

for j in range(n_pairs, rows * cols):
    r = j // cols
    c = j % cols
    axes[r, c].set_visible(False)

plt.tight_layout()
plt.show()

print(f"\nOne-Class SVM (UNAS):")
print(f"  • Kernel : {final_model.kernel}")
print(f"  • Gamma  : {gamma_val:.2e}")
print(f"  • Nu     : {nu_val}")
print(f"  • Candidatos UNAS (outliers): {np.sum(outliers_mask_unas)}/"
      f"{len(X_unas_scaled)}  ({100*np.sum(outliers_mask_unas)/len(X_unas_scaled):.2f}%)")


In [ ]:

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Inliers UNAS (azul claro)
inliers_mask_unas = (unas_preds == 1)
if np.any(inliers_mask_unas):
    ax.scatter(
        X_unas_scaled[inliers_mask_unas, 0],
        X_unas_scaled[inliers_mask_unas, 1],
        X_unas_scaled[inliers_mask_unas, 2],
        c='orange', s=20, alpha=0.5,
        label=f'UNAS Inliers: {np.sum(inliers_mask_unas)}'
    )

# Outliers UNAS (rojo oscuro)
outliers_mask_unas = (unas_preds == -1)
if np.any(outliers_mask_unas):
    ax.scatter(
        X_unas_scaled[outliers_mask_unas, 0],
        X_unas_scaled[outliers_mask_unas, 1],
        X_unas_scaled[outliers_mask_unas, 2],
        c='darkred', marker='^', s=80, alpha=0.9,
        label=f'UNAS Outliers: {np.sum(outliers_mask_unas)}'
    )

# Etiquetas y título
ax.set_xlabel('Alpha (escalado)')
ax.set_ylabel('Beta (escalado)')
ax.set_zlabel('Flux (escalado)')
gamma_val = final_model.gamma if hasattr(final_model, 'gamma') else final_model._gamma
nu_val = final_model.nu
ax.set_title(f"3D UNAS Predictions\nν={nu_val}, γ={gamma_val:.2e}")
ax.legend(loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
